# The Sed object

This example demonstrates the various methods associated with the `Sed` class.

`Sed` objects can be extracted directly from [Grid objects](../grids/grids.rst) or created by [Galaxy objects](../galaxy/galaxy.rst).

We start by loading some modules and initialising a grid:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from unyt import Angstrom, Hz, erg, eV, s, um

from synthesizer.emission_models.attenuation import Madau96
from synthesizer.emissions import Sed, plot_spectra_as_rainbow
from synthesizer.filters import FilterCollection
from synthesizer.grid import Grid

grid_dir = "../../../tests/test_grid/"
grid_name = "test_grid"
# Let's load a grid and truncate it to a wavelength range
grid = Grid(
    grid_name, grid_dir=grid_dir, lam_lims=[400 * Angstrom, 1e5 * Angstrom]
)

Next, let's extract the spectra at a given point in our grid.
We first define a target age and metallicity, obtain the index of the grid at these values, then extract the spectra.
This will be in the form of an ``Sed`` object.

In [ ]:
log10age = 6.0  # log10(age/yr)
metallicity = 0.01
spectra_type = "incident"
grid_point = grid.get_grid_point(log10ages=log10age, metallicity=metallicity)
sed = grid.get_sed_at_grid_point(grid_point, spectra_type=spectra_type)
sed.lnu *= 1e8  # multiply initial stellar mass

print(sed)

`Sed` objects contain a wavelength grid and luminosity in the `lam` and `lnu` attributes. Both come with units making them easy to convert:

In [ ]:
print(sed.lam)
print(sed.lnu)

These also have more descriptive aliases:

In [ ]:
print(sed.wavelength)
print(sed.luminosity_nu)

Thus we can easily make a plot:

In [ ]:
plt.plot(np.log10(sed.lam), np.log10(sed.lnu))
plt.show()

We can also also visualise the spectrum as a rainbow.

In [ ]:
fig, ax = plot_spectra_as_rainbow(sed)
plt.show()

We can also get the luminosity ($L$) or spectral luminosity density per $\AA$ ($L_{\lambda}$): 

In [ ]:
print(sed.luminosity)
print(sed.llam)

``Sed`` objects can be easily scaled via the `*` operator. For example,

In [ ]:
sed_x_10 = sed * 10

plt.plot(np.log10(sed.lam), np.log10(sed.lnu))
plt.plot(np.log10(sed_x_10.lam), np.log10(sed_x_10.lnu))
plt.show()

## Methods

There are a number of helper methods on ``Sed`` for calculating common derived properties. We provide some examples below.

We can calculate the integrated bolometric luminosity of the Sed using the `bolometric_luminosity` property method:

In [ ]:
sed.bolometric_luminosity

By default any spectra integration will use a trapezoid method. It's also possible to use the simpson rule using the `measure_bolometric_luminosity` method.

In [ ]:
sed.measure_bolometric_luminosity(integration_method="simps")

You can also get the luminosity or `lnu` in a particular window by passing the wavelengths defining the limits of the window.

In [ ]:
sed.measure_window_luminosity((1400.0 * Angstrom, 1600.0 * Angstrom))

Notice how units were provided with this window. These are required but also allow you to use an arbitrary unit system.

In [ ]:
sed.measure_window_luminosity((0.14 * um, 0.16 * um))

In [ ]:
sed.measure_window_lnu((1400.0 * Angstrom, 1600.0 * Angstrom))

As for the bolometric luminosity, there are multiple integration methods that can be used for calculating the window.
By default synthesizer will use the trapezoid method (`"trapz"`), but you can also take a simple average.

In [ ]:
sed.measure_window_lnu(
    (1400.0 * Angstrom, 1600.0 * Angstrom), integration_method="average"
)

We can measure a spectral break by providing two windows:

In [ ]:
sed.measure_break((3400, 3600) * Angstrom, (4150, 4250) * Angstrom)

There are also a few in-built break methods, e.g. `measure_Balmer_break()`

In [ ]:
sed.measure_balmer_break()

In [ ]:
sed.measure_d4000()

We can also measure absorption line indices, and spectral slopes (e.g. the UV spectral slope $\beta$):

In [ ]:
sed.measure_index(
    (1500, 1600) * Angstrom, (1400, 1500) * Angstrom, (1600, 1700) * Angstrom
)

In [ ]:
sed.measure_beta()

By default this uses a single window and fits the spectrum by a power-law. However, we can also specify two windows as below, in which case the luminosity in each window is calcualted and used to infer a slope:

In [ ]:
sed.measure_beta(window=(1250, 1750, 2250, 2750) * Angstrom)

Finally, we can also measure ionising photon production rates at a particular ionisation energy:

In [ ]:
sed.calculate_ionising_photon_production_rate(
    ionisation_energy=13.6 * eV, limit=1000
)

## Observed frame SED

By default ``Sed`` objects are defined in the rest frame. 
To move an SED to the observer frame we need to provide a cosmology, using an `astropy.cosmology` object, a redshift $z$, and optionally an IGM absorption model (see [here](../attenuation/igm.ipynb) for details).

In [ ]:
from astropy.cosmology import Planck18 as cosmo

z = 3.0  # redshift
sed.get_fnu(cosmo, z, igm=Madau96)  # generate observed frame spectra

plt.plot(np.log10(sed.obslam), np.log10(sed.fnu))
plt.show()

## Photometry

Once we have computed the observed frame SED there is a method on an `Sed` object that allows us to calculate observed photometry (the same is of course true for rest frame photometry). However, first we need to instantiate a `FilterCollection` object (see [Filters](../filters/filters.rst) for details):

In [ ]:
filter_codes = [
    f"JWST/NIRCam.{f}"
    for f in [
        "F070W",
        "F090W",
        "F115W",
        "F150W",
        "F200W",
        "F277W",
        "F356W",
        "F444W",
    ]
]  # define a list of filter codes
fc = FilterCollection(filter_codes, new_lam=grid.lam)

# Measure observed photometry
fluxes = sed.get_photo_fnu(fc)
print(fluxes)

## Multiple SEDs

An ``Sed`` object can be composed of multiple SEDs ,if their dimensions are identical (i.e. same wavelength resolution). This is common when we are, for example, modelling the spectra of particle based galaxy.

To demonstrate, let's create an `Sed` object with two seds:

In [ ]:
sed2 = Sed(sed.lam, np.array([sed.lnu, sed.lnu * 2]) * erg / s / Hz)
print(sed2.lnu.shape)

print(sed2.measure_window_lnu((1400, 1600) * Angstrom))
print(sed2.measure_beta())
print(sed2.measure_beta(window=(1250, 1750, 2250, 2750) * Angstrom))
print(sed2.measure_balmer_break())
print(
    sed2.measure_index(
        (1500, 1600) * Angstrom,
        (1400, 1500) * Angstrom,
        (1600, 1700) * Angstrom,
    )
)
print(
    sed2.calculate_ionising_photon_production_rate(
        ionisation_energy=13.6 * eV, limit=1000
    )
)

This even works for N-dimensional spectra. For example, if we convert an entire grid into a set of spectra:

In [ ]:
sed3 = grid.get_sed(spectra_type="incident")
print(sed3.lnu.shape)

print(sed3.measure_window_lnu((1400, 1600) * Angstrom))
print(
    sed3.calculate_ionising_photon_production_rate(
        ionisation_energy=13.6 * eV, limit=1000
    )
)

## Combining SEDs

``Sed`` objects can be combined either via concatenation, to produce a single ``Sed`` holding multiple spectra, or by addition, to add the spectra contained in the input ``Sed`` objects. 

To concatenate spectra we use ``Sed.concat()``. 
`Sed.concat` can take an arbitrary number of `Sed` objects to combine.

In [ ]:
print("Shapes before:", sed._lnu.shape, sed2._lnu.shape)
sed3 = sed2.concat(sed)
print("Combined shape:", sed3._lnu.shape)

sed4 = sed2.concat(sed, sed2, sed3)
print("Combined shape:", sed4._lnu.shape)

If we want to add the spectra of two ``Sed`` objects together, we simply apply the `+` operator.
However, unlike `concat`, this will only work for ``Sed``s with identical shapes.

In [ ]:
sed_add = sed + sed
plt.plot(np.log10(sed.lam), np.log10(sed.lnu), label="sed")
plt.plot(np.log10(sed_add.lam), np.log10(sed_add.lnu), label="sed5")
plt.ylim(26, 30)
plt.xlim(2.8, 4.5)
plt.xlabel(r"$\rm log_{10}(\lambda/\AA)$")
plt.ylabel(r"$\rm log_{10}(L_{\nu}/erg\ s^{-1}\ Hz^{-1} M_{\odot}^{-1})$")
plt.legend()
plt.show()
plt.close()

## Resampling SEDs

The `Sed` includes a method to resample an sed, e.g. to lower-resolution or to match some target resolution (e.g. from observations).

In [ ]:
sed_resampled = sed.get_resampled_sed(5)
plt.plot(np.log10(sed.lam), np.log10(sed.lnu), label="Original")
plt.plot(
    np.log10(sed_resampled.lam), np.log10(sed_resampled.lnu), label="Resampled"
)
plt.xlim(2.8, 3.5)
plt.ylim(27.0, 29.5)
plt.xlabel(r"$\rm log_{10}(\lambda/\AA)$")
plt.ylabel(r"$\rm log_{10}(L_{\nu}/erg\ s^{-1}\ Hz^{-1} M_{\odot}^{-1})$")
plt.legend()
plt.show()
plt.close()

In [ ]:
print(
    sed.measure_bolometric_luminosity() / sed3.measure_bolometric_luminosity()
)

## Applying attenuation

To apply attenuation to an `Sed` you can use the `apply_attenuation` method and pass the optical depth and a dust curve (see [Attenuation](../attenuation/dust_attenuation.ipynb) for more details on dust curves).

In [ ]:
from synthesizer.emission_models.attenuation import PowerLaw

sed4_att = sed4.apply_attenuation(tau_v=0.7, dust_curve=PowerLaw(-1.0))

plt.plot(np.log10(sed4.lam), np.log10(sed4.lnu[0, :]), label="Incident")
plt.plot(
    np.log10(sed4_att.lam), np.log10(sed4_att.lnu[0, :]), label="Attenuated"
)
plt.xlim(2.8, 3.5)
plt.ylim(26.0, 30.0)
plt.xlabel(r"$\rm log_{10}(\lambda/\AA)$")
plt.ylabel(r"$\rm log_{10}(L_{\nu}/erg\ s^{-1}\ Hz^{-1} M_{\odot}^{-1})$")
plt.legend()
plt.show()
plt.close()

``apply_attenuation`` can also accept a ``mask``, which applies attenuation to specific spectra in a multidimensional ``Sed`` (like an `Sed` containing the spectra for multiple particles, for instance.)

In [ ]:
sed0_att = sed4.apply_attenuation(
    tau_v=0.7,
    dust_curve=PowerLaw(-1.0),
    mask=np.array([0, 1, 0, 0, 0, 0, 0, 0], dtype=bool),
)

plt.plot(np.log10(sed4.lam), np.log10(sed4.lnu[1, :]), label="Incident")
plt.plot(np.log10(sed4.lam), np.log10(sed0_att.lnu[0, :]), label="Attenuated")
plt.xlim(2.6, 3.5)
plt.ylim(26.0, 30.0)
plt.xlabel(r"$\rm log_{10}(\lambda/\AA)$")
plt.ylabel(r"$\rm log_{10}(L_{\nu}/erg\ s^{-1}\ Hz^{-1} M_{\odot}^{-1})$")
plt.legend()
plt.show()
plt.close()

## Calculating transmission

If you have an attenuated SED, a natural quantity to calculate is the extinction of that spectra ($A$). This can be done either at the wavelengths of the `Sed`, an arbitrary wavelength/wavelength array, or at commonly used values (1500 and 5500 angstrom) using functions available in the `sed` module. Note that these functions return the extinction/attenuation in magnitudes. Below is a demonstration.

In [ ]:
from unyt import angstrom, um

from synthesizer.emissions import (
    get_attenuation,
    get_attenuation_at_1500,
    get_attenuation_at_5500,
    get_attenuation_at_lam,
)

# Get an intrinsic spectra
grid_point = grid.get_grid_point(log10ages=7, metallicity=0.01)
int_sed = grid.get_sed_at_grid_point(grid_point, spectra_type="incident")

# Apply an attenuation curve
att_sed = int_sed.apply_attenuation(tau_v=0.7, dust_curve=PowerLaw(-1.0))

# Get attenuation at sed.lam
attenuation = get_attenuation(int_sed, att_sed)

# Get attenuation at 5 microns
att_at_5 = get_attenuation_at_lam(5 * um, int_sed, att_sed)

# Get attenuation at an arbitrary range of wavelengths
att_at_range = get_attenuation_at_lam(
    np.linspace(5000, 10000, 5) * angstrom, int_sed, att_sed
)

# Get attenuation at 1500 angstrom
att_at_1500 = get_attenuation_at_1500(int_sed, att_sed)

# Get attenuation at 5500 angstrom
att_at_5500 = get_attenuation_at_5500(int_sed, att_sed)